# 2. Train data augmentation

## Resumen

Este cuaderno representa la segunda etapa del proyecto. Cómo el objetivo es desarrollar diferentes modelos que sean capaces de clasificar si una imagen contiene o no la mosca de la fruta, **realizaremos previamente un aumento de los datos de entrenamiento** de tal manera que todos los modelos hagan uso del mismo *dataset*.

Por lo tanto en este cuaderno el objetivo es,
1. Dividir el conjunto de datos en *train/test*.
2. Realizar y guardar el aumento de las imágenes de entrenamiento.

## Código

### Dependencias

Las versiones empleadas en el cuaderno han sido:
* Python version: 3.11.3
* NumPy version: 1.23.5
* OpenCV version: 4.7.0
* Albumentations version: 1.3.0

In [1]:
!python --version

Python 3.11.3


In [2]:
import cv2
import numpy
import albumentations

print("NumPy version:", numpy.__version__)
print("OpenCV version:", cv2.__version__)
print("Albumentations version:", albumentations.__version__)

NumPy version: 1.23.5
OpenCV version: 4.7.0
Albumentations version: 1.3.0


### Librerías

In [3]:
import os
import random

In [4]:
import numpy as np
import cv2

In [5]:
import albumentations as A

### Funciones

* **data_restruct:** 
* **data_augment:**

In [17]:
def data_restruct(path, seed):
    # Lista de etiquetas
    labels = os.listdir(path)
    # Cargar las imágenes y las etiquetas
    X, y, names = [], [], []
    for i, label in enumerate(labels):
        img_names = os.listdir(os.path.join(path, label))
        for img_name in img_names:
            img_path = os.path.join(path, label, img_name)
            img = cv2.imread(img_path)
            if img.shape[0] != 32 or img.shape[1] != 32:
                print(img_name, label)
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            X.append(img)
            y.append(i)
            names.append(img_name)
            
    # Mezclar las listas
    combinadas = list(zip(X, y, names))
    if seed is not None:
        random.seed(seed)
    random.shuffle(combinadas)
    # Desempaquetar las listas mezcladas en dos listas separadas
    X, y, names = zip(*combinadas)
            
    return np.array(X), np.array(y), np.array(names), labels

In [7]:
def data_augment(img, transform, n=5, n_fixed = True):
    """
    Función para llevar a cabo aumento de las imágenes.
    Inputs:
        · img (np.array): Imagen a transformar.
        · transform (Albumentations object): Pipeline con las posibles acciones a llevar en la imagen.
        · n (int)
        · n_fixed (bool): Define si n es la cantidad de transformaciones a realizar o la cantidad de veces que se intenta.
    Outputs:
        · - (np.array): Array con las transformaciones realizadas.
    """
    res, counter = [], 0
    while True:
        transf = transform(image=img)['image']
        # Si la transf. resultante es diferente del original y diferente a las nuevas alteraciones
        if not np.array_equal(img, transf) and not np.isin(transf, res).all():
            res.append(transf)
        if (n_fixed == True and len(res) >= n) or (n_fixed == False and counter >= n) or (n_fixed == False and len(res) >= n):
            return np.stack(res, axis=0)
        
        counter += 1
        
def data_augment_validation(augmentations):
    """
    Función para validar que el proceso de aumento de datos ha ido bien.
    1. comprobar si ha habido al menos una alteración de la imagen. Este factor realmente no es restrictivo,
    simplemente queremos controlarlo.
    2. Comprobar que no se ha colado ninguna imagen repetida
    """
    # Comprobamos si NO se ha realizado ninguna transformación
    if augmentations.shape[0] == 0:
        print(name)
        print('ups')
    # Validamos que no hay arrays (imgs) repetidos en la lista
    unique, counts = np.unique(np.array(list(augmentations)), axis=0, return_counts=True)
    if len(unique[counts > 1]) > 0:
        print(name)
        print('Arrays repetidos:', repeated)

### Main pipeline

El primer paso será importar el conjunto de datos que los propios **zoólogos del grupo ZAP de la UIB** han etiquetado ayudándonos de la función `data_restruct()`.

In [86]:
folder_path = r'C:\Users\migue\OneDrive - Universitat de les Illes Balears\Proyectos\ZAP\FruitFlyNet\scripts\dataset\original'
X, y, names, labels = data_restruct(folder_path, seed=1)

print(' ·X:',X.shape)
print(' ·y:',y.shape)
print(' ·names:',names.shape)
print(' ·labels:',len(labels))

 ·X: (84, 32, 32, 3)
 ·y: (84,)
 ·names: (84,)
 ·labels: 4


In [87]:
labels

['fly', 'frame', 'small black bug', 'trap']

In [88]:
names[0]

'230518185338.png'

Observamos cuantos datos tenemos para cada etiqueta.

In [89]:
unique, counts = np.unique(y, return_counts=True)
for la, co in zip(labels, counts):
    print(f'{la}: {co}')

fly: 21
frame: 21
small black bug: 21
trap: 21


Ya tenemos los datos de trabajo, **dividimos los datos en *train/test***.
* 80% para *train*.
* 20% para *test*.

In [90]:
TRAIN_PER = 0.8

In [91]:
train_len = int(X.shape[0] * TRAIN_PER)
X_train, X_test = np.split(X, [train_len])
y_train, y_test = np.split(y, [train_len])
names_train, names_test = np.split(names, [train_len])

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(67, 32, 32, 3)
(67,)
(17, 32, 32, 3)
(17,)


Una vez tenemos los datos de trabajo, **realizamos el proceso de aumento del *dataset***. Para ello primero definimos 
* cuantas imágenes nuevas queremos generar,
* si este valor es fijo o no
* y las transformadas a realizar junto a la probabilidad de que estas sucedan.

In [92]:
N, N_FIXED = 10, False

transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
])

Realizamos el proceso para cada una de las imágenes. Las nuevas imágenes tendrán la etiqueta `augmented` en su nombre para poder diferenciarlas de la original.

In [93]:
for img_index in range(X_train.shape[0]):
    # Obtenemos la imagen y etiqueta a trabajar y aplicamos data augmentation
    img, target, name = X_train[img_index], y_train[img_index], names_train[img_index]
    augmentations = data_augment(img, transform, n=N, n_fixed=N_FIXED)
    # Validación del aumento de datos
    data_augment_validation(augmentations)
    # Añadimos a nuestro dataset las imágenes aumentadas, etiquetas y nombres
    X_train = np.concatenate((X_train, augmentations), axis=0)
    y_train = np.append(y_train, np.full(augmentations.shape[0], target))
    name = name.split('.')
    name, ext = name[0], name[1]
    full_names = [name+'_augmented_'+str(i+1)+'.'+ext for i in range(augmentations.shape[0])]
    names_train = np.append(names_train, full_names)
        
print(X_train.shape)
print(y_train.shape)
print(names_train.shape)

(445, 32, 32, 3)
(445,)
(445,)


Finalmente guardamos nuestro nuevo *dataset* que contiene las imágenes así cómo las diferentes versiones.

In [94]:
# Mezclamos el conjunto de datos
combinadas = list(zip(X_train, y_train, names_train))
random.shuffle(combinadas)
X_train, y_train, names_train = zip(*combinadas)

# Comprobamos (y/o creamos) si la carpeta destino existe
path = 'C:/Users\migue/OneDrive - Universitat de les Illes Balears/Proyectos/ZAP/FruitFlyNet/scripts/dataset/'
folder = 'augmented_dataset'
full_path = os.path.join(path, folder)
try: os.mkdir(full_path)
except FileExistsError: pass

# Creamos la carpeta de train
full_train_path = os.path.join(full_path, 'train')
try: os.mkdir(full_train_path)
except FileExistsError: pass
for i,val in enumerate(zip(X_train, y_train, names_train)):
    # Desempaquetamos y generamos la ruta
    img, target, name = val
    label_path = os.path.join(full_train_path, labels[target])
    # Validamos si ya existe la carpeta
    try: os.mkdir(label_path)
    except FileExistsError: pass
    # Guardamos la imagen
    final_path = os.path.join(label_path, name)
    cv2.imwrite(final_path, cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    
    
# Creamos la carpeta de test
full_test_path = os.path.join(full_path, 'test')
try: os.mkdir(full_test_path)
except FileExistsError: pass
for i,val in enumerate(zip(X_test, y_test, names_test)):
    # Desempaquetamos y generamos la ruta
    img, target, name = val
    label_path = os.path.join(full_test_path, labels[target])
    # Validamos si ya existe la carpeta
    try: os.mkdir(label_path)
    except FileExistsError: pass
    # Guardamos la imagen
    final_path = os.path.join(label_path, name)
    cv2.imwrite(final_path, cv2.cvtColor(img, cv2.COLOR_BGR2RGB))